In [422]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [423]:
%env twitter_api_key

'pFlQlVe3RxDDloIFv3Wq3lv2I'

In [424]:
# criar um dataset com dados de usuários do twitter + seus x tweets.

In [425]:
# packages imports here
import tweepy
import pandas as pd
import numpy as np
from easydict import EasyDict as edict

In [475]:
# Loading variables for tweepy

twitter = {}
twitter['TWITTER_API_KEY'] = %env twitter_api_key 
twitter['TWITTER_API_SECRET'] = %env twitter_api_secret
twitter['TWITTER_API_TOKEN'] = %env twitter_access_token
twitter['TWITTER_API_TOKEN_SECRET'] = %env twitter_access_token_secret


#for key in twitter:
#    print(key, '->', twitter[key])

# Authenticate with Twitter API

twitter['TWITTER_API_TOKEN_SECRET']
auth = tweepy.OAuthHandler(twitter['TWITTER_API_KEY'], twitter['TWITTER_API_SECRET'])

# se não autenticar com o access token algumas funcionalidades da api 
# não ficam disponíveis. tipo verificar rate limits.
auth.set_access_token(twitter['TWITTER_API_TOKEN'], twitter['TWITTER_API_TOKEN_SECRET'])

# o objeto api é utilizado para realizar toda comunicação com a API do twitter.
api = tweepy.API(auth)

In [515]:
# recuperando dados de um twitter user
def getDataFromUserByTwiiterHandle(handle):
    try:
        user = api.get_user(screen_name = handle)
        return user
    except tweepy.TweepError as e:
#         print("Tweepy Error retrieving user: {}".format(e))
        return e
#         return {'reason' : e.reason, 'codes' : e.api_code, 'response' : e.response, 'args' : e.args}

In [562]:
# recuperando dados de um twitter user
def getUserTimeline(uid):
    try:
        timeline = api.user_timeline(user_id = uid, exclude_replies=True, include_rts=True)
        return timeline
    except tweepy.TweepError as e:
#         print("Tweepy Error timeline: {}".format(e))
        return {'reason' : e.reason, 'codes' : e.api_code, 'response' : e.response, 'args' : e.args}

In [429]:
# create user dataframe by list of users retrieved by tweepy
def createDataFrameFromUser(user, dfUsers, error=False, handle=None):
#     dfUsers = pd.DataFrame();
#     for user in users:
    if error == False:
        u = {
            'twitter_id': user.id,
            'name': user.name,
            'handle' : user.screen_name,
            'twitter_is_protected': user.protected,
            'description': user.description,
            'followers_count': user.followers_count,
            'friends_count': user.friends_count,
            'location': user.location,
            'created_at': user.created_at,
            'verified': user.verified,
            'lang': user.lang,
            'default_profile': user.default_profile,
            'profile_image': user.profile_image_url,
            'withheld_in_countries' : user.withheld_in_countries
        }
        dfUsers = dfUsers.append(u, ignore_index=True)
    else:
        e = {'error': user, 'handle' : handle}
        dfUsers = dfUsers.append(e, ignore_index=True)
    return dfUsers

In [430]:
def createDataFrameFromTimeline(timeline, dfTimeline, error = False, handle=None):
    if error == False:
        for tweet in timeline:
            t = {
                'tweet_id' : tweet.id,
                'tweet_author': tweet.author.screen_name,
                'tweet_id_str' : tweet.id_str,
                'tweet_retweeted' : tweet.retweeted,
                'tweet_created_at': tweet.created_at,
                'tweet_source' : tweet.source,
                'tweet_author_id_str':tweet.author.id_str,
                'tweet_text':tweet.text,
                'tweet_contributors': tweet.contributors,
                'tweet_favorite_count':tweet.favorite_count,
                'tweet_favorited' : tweet.favorited,
                'tweet_geo' : tweet.geo,
                'tweet_is_retweet': tweet.is_quote_status,
                'tweet_lang' : tweet.lang,
                'tweet_place' : tweet.place,
                'tweet_hashtags' : tweet.entities.get('hashtags')
            }
            dfTimeline = dfTimeline.append(t, ignore_index=True)
    else:
        e = {'error': timeline, 'handle' : handle}
        dfTimeline = dfTimeline.append(e, ignore_index=True)
    return dfTimeline    

In [538]:
twitterUser = getDataFromUserByTwiiterHandle("Eustacia251")
#print(twitterUser)
print(twitterUser.api_code)
print(type(twitterUser))


# response = getUserTimeline(twitterUser.id_str)
# for tweet in response:
#     print(tweet.text)
#     print(tweet.author.screen_name)
#     print(tweet.is_quote_status)

63
<class 'tweepy.error.TweepError'>


In [ ]:
handlescsv = pd.read_csv("/Users/dc/Downloads/cristian.csv",
                      delimiter=",", encoding="latin")
handlescsv = handlescsv['Perfil']

handlescsv.info()

In [568]:
# dataframes creation

columns = {
    'error'
}

dfUser = pd.DataFrame(columns=columns);
dfTimelines = pd.DataFrame(columns=columns);

#for handle in handles:
#handles = {'blckjxxxxxxzz', 'VieiraClever1', 'blckjzz'} # testing handles {do not exist, blocked timeline, real}

count = 0
stop = 300
for handle in handlescsv:
    if count > stop:
        break
    count +=1
    print(f'{count} # Analisando o perfil:@ {handle}')
    twitterUser = getDataFromUserByTwiiterHandle(handle)
    # if its true, the twitter api has returned some error
    # False, should mean that the user is found and you can work with it.
#     print(type(twitterUser))
    if isinstance(twitterUser, tweepy.error.TweepError) == False: # means that the user exists because the Twitter API response
        print(f'Encontrei o {handle}. Seu nome é: {twitterUser.name}')
        # When you have the user retrieved
        # The Expected flow should be written here
        dfUser = createDataFrameFromUser(twitterUser, dfUser)
        if twitterUser.protected == False: #means that the timeline is open and you should be allowed to retrieve data
            response = getUserTimeline(twitterUser.id_str)
            # all analysed user will be stored to the same dataframe
            # and its tweets will be agragated into a dataframe within all users tweets retrived.
            dfTimelines = createDataFrameFromTimeline(response, dfTimelines) # should add data to the main dataframe
        else: # timeline closed
            response = getUserTimeline(twitterUser.id_str)
            print(f'{count} Timeline de :@ {handle} é fechada!')
            dfTimelines = createDataFrameFromTimeline(response.get('reason'), dfTimelines, error=True, handle=handle) # should add data to the main dataframe
            # throw error.
            # log error in the table
            # move to the next line
#             break    
    else:
        print(f'{count} O Usuário @{handle} não rolou. Razão: {twitterUser.reason}')
        dfUser = createDataFrameFromUser(twitterUser.reason, dfUser, error=True, handle = handle)
        # move to the next line of the handle dataset
        # log some error into the dataset
        # continue the flux
        continue  
        
  
    

1 # Analisando o perfil:@ Eustacia251
1 O Usuário @Eustacia251 não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
2 # Analisando o perfil:@ VieiraClever1
Encontrei o VieiraClever1. Seu nome é: Clever Vieira🌹💚💛🇧🇷🇧🇷🌹
2 Timeline de :@ VieiraClever1 é fechada!
3 # Analisando o perfil:@ carolin69045611
3 O Usuário @carolin69045611 não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
4 # Analisando o perfil:@ shenybert
4 O Usuário @shenybert não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
5 # Analisando o perfil:@ VavaDone
5 O Usuário @VavaDone não rolou. Razão: [{'code': 50, 'message': 'User not found.'}]
6 # Analisando o perfil:@ AlziraRosa10
6 O Usuário @AlziraRosa10 não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
7 # Analisando o perfil:@ de_novaki
Encontrei o de_novaki. Seu nome é: Rosângela de Souza novaki
7 Timeline de :@ de_novaki é fechada!
8 # Analisando o perfil:@ jorgebrito798
8 O Usuário @jor

61 O Usuário @erimmaxwell não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
62 # Analisando o perfil:@ Titiacila75
62 O Usuário @Titiacila75 não rolou. Razão: [{'code': 50, 'message': 'User not found.'}]
63 # Analisando o perfil:@ JROBERTOBtt
63 O Usuário @JROBERTOBtt não rolou. Razão: [{'code': 50, 'message': 'User not found.'}]
64 # Analisando o perfil:@ Brilhan46288780
64 O Usuário @Brilhan46288780 não rolou. Razão: [{'code': 50, 'message': 'User not found.'}]
65 # Analisando o perfil:@ CeliaMa02424950
65 O Usuário @CeliaMa02424950 não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
66 # Analisando o perfil:@ Alexis23394668
66 O Usuário @Alexis23394668 não rolou. Razão: [{'code': 50, 'message': 'User not found.'}]
67 # Analisando o perfil:@ Roxi1058
67 O Usuário @Roxi1058 não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
68 # Analisando o perfil:@ Magaly40119384
68 O Usuário @Magaly40119384 não rolou. Razão: [{'code': 6

122 O Usuário @LimaEvio não rolou. Razão: [{'code': 50, 'message': 'User not found.'}]
123 # Analisando o perfil:@ LuisMen61860009
123 O Usuário @LuisMen61860009 não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
124 # Analisando o perfil:@ Rosnier14
124 O Usuário @Rosnier14 não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
125 # Analisando o perfil:@ PaulaLi01322083
125 O Usuário @PaulaLi01322083 não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
126 # Analisando o perfil:@ Fernan_Olivier
126 O Usuário @Fernan_Olivier não rolou. Razão: [{'code': 50, 'message': 'User not found.'}]
127 # Analisando o perfil:@ EmilisVentimil1
127 O Usuário @EmilisVentimil1 não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
128 # Analisando o perfil:@ DeniShelbyY
Encontrei o DeniShelbyY. Seu nome é: Denilson ᵇᶠʳ★彡
128 Timeline de :@ DeniShelbyY é fechada!
129 # Analisando o perfil:@ Deimarysmartine
129 O Usuário @Deimarys

182 O Usuário @RaquelBuenoQue1 não rolou. Razão: [{'code': 50, 'message': 'User not found.'}]
183 # Analisando o perfil:@ MermejoAngel
183 O Usuário @MermejoAngel não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
184 # Analisando o perfil:@ _ElonMuska
184 O Usuário @_ElonMuska não rolou. Razão: [{'code': 50, 'message': 'User not found.'}]
185 # Analisando o perfil:@ JVarenzuela
185 O Usuário @JVarenzuela não rolou. Razão: [{'code': 50, 'message': 'User not found.'}]
186 # Analisando o perfil:@ erasmo97754623
186 O Usuário @erasmo97754623 não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
187 # Analisando o perfil:@ KanitzFlavio
187 O Usuário @KanitzFlavio não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
188 # Analisando o perfil:@ LUISMER08539169
188 O Usuário @LUISMER08539169 não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
189 # Analisando o perfil:@ celsogarciacom
189 O Usuário @celsogarciacom n

241 O Usuário @franklinzorril9 não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
242 # Analisando o perfil:@ Eliannyta1
242 O Usuário @Eliannyta1 não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
243 # Analisando o perfil:@ OniolysV
243 O Usuário @OniolysV não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
244 # Analisando o perfil:@ luainny1105
244 O Usuário @luainny1105 não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
245 # Analisando o perfil:@ Josepacheco213
245 O Usuário @Josepacheco213 não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
246 # Analisando o perfil:@ Antonio39812268
246 O Usuário @Antonio39812268 não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
247 # Analisando o perfil:@ NestorPiamo
247 O Usuário @NestorPiamo não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
248 # Analisando o perfil:@ VilmaOlivares9
248 O Usuário 

300 O Usuário @Yenibel111 não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]
301 # Analisando o perfil:@ JolbertRomero1
301 O Usuário @JolbertRomero1 não rolou. Razão: [{'code': 63, 'message': 'User has been suspended.'}]


In [571]:
dfUser.head()

,error,handle,created_at,default_profile,description,followers_count,friends_count,lang,location,name,profile_image,twitter_id,twitter_is_protected,verified,withheld_in_countries
0,"[{'code': 63, 'message': 'User has been suspen...",Eustacia251,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,VieiraClever1,2017-08-14 22:05:05,1.0,Sonhos não envelhecem...,504.0,987.0,None,MG Brasil,Clever Vieira🌹💚💛🇧🇷🇧🇷🌹,http://pbs.twimg.com/profile_images/1427372835...,8.972175e+17,1.0,0.0,[]
2,"[{'code': 63, 'message': 'User has been suspen...",carolin69045611,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'code': 63, 'message': 'User has been suspen...",shenybert,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[{'code': 50, 'message': 'User not found.'}]",VavaDone,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [573]:
dfUser.to_csv("/Users/dc/Downloads/df_users.csv", sep=',', encoding="utf-8")

In [574]:
dfTimelines.to_csv("/Users/dc/Downloads/df_timelines.csv", sep=',', encoding="utf-8")

In [570]:
dfTimelines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   error                 14 non-null     object        
 1   handle                14 non-null     object        
 2   tweet_author          116 non-null    object        
 3   tweet_author_id_str   116 non-null    object        
 4   tweet_contributors    0 non-null      object        
 5   tweet_created_at      116 non-null    datetime64[ns]
 6   tweet_favorite_count  116 non-null    float64       
 7   tweet_favorited       116 non-null    object        
 8   tweet_geo             0 non-null      object        
 9   tweet_hashtags        116 non-null    object        
 10  tweet_id              116 non-null    float64       
 11  tweet_id_str          116 non-null    object        
 12  tweet_is_retweet      116 non-null    object        
 13  tweet_lang          